# 分析不合规合同

基本思路: 读取每周的统计数据, 生成一个全量的数据集存入本地 Excel 中, 然后根据全量的数据集统计数据做数据分析.


In [1]:
from functools import partial
from pathlib import Path
import pandas as pd
pd.options.display.max_rows = 5


将周统计 Excel 文件中的多个 sheet 映射成内部定义的名称, 实现对周统计 Excel 的解耦.

In [6]:
# config
sheet_names = ['倒签（审批通过）',
               '倒签（审批中）',
               '2022年所有合同',
               '应结未结（已终止）',
               '应结未结（审批中）',
               '应结未结（未提交）',
               '应结未结（已终止未结算）',
               '倒签&应结未结汇总',
               '字典',
               '产园项目清单',
               '所有组织机构',
               '分公司+项目部+项目'
               ]
name_for_df = ['reverse_a',
               'reverse_b',
               '2022',
               'should_end_a',
               'should_end_b',
               'should_end_c',
               'should_end_d',
               'all',
               'dict',
               'projects',
               'org',
               'org2'
               ]

df_sheets = pd.DataFrame({
    'sheet_name': sheet_names,
    'df_name': name_for_df
})

df_sheets


,sheet_name,df_name
0,倒签（审批通过）,reverse_a
1,倒签（审批中）,reverse_b
...,...,...
10,所有组织机构,org
11,分公司+项目部+项目,org2


获取最新的周统计 Excel 文件的句柄.

In [8]:
# get or create 'data' directory
data_dir_name = 'data'
data_dir = Path.cwd() / data_dir_name
if not data_dir.exists():
    data_dir.mkdir()
# get all non-compliant contracts xlsx files into a array
non_compliant_contracts_dir = data_dir / 'non-compliant_contracts'
files = [f for f in sorted(non_compliant_contracts_dir.glob('20*.xlsx'))]
# tp: this period 当期的意思
tp_filename = files[-1]
tp_filename

PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts/20220318.xlsx')

将最新的周统计即本周期的 Excel 文件中的 sheets 映射成 DataFrame.

In [9]:
def map_sheet_name(sheet_name_, df_, lookup, target):
    """Map sheet names in Excel file
    
    Parameters:
    ----------
    sheet_name_: str
      sheet name in Excel file
    df_: DataFrame
      mapping relations
    lookup: str
      filed name in DataFrame (df_) that is sheet name in excel file
    target: str
      map to name

    Returns:
    -------
    name: str
    """
    values = df_.loc[df_[lookup] == sheet_name_][target].values
    if len(values) > 0:
        return values[0]
    return sheet_name_


map_name = partial(map_sheet_name, df_=df_sheets,
                   lookup='sheet_name', target='df_name')


In [12]:
tp_dfs = {}
xls = pd.ExcelFile(tp_filename)
for sheet_name in xls.sheet_names:
    tp_dfs[map_name(sheet_name)] = pd.read_excel(xls, sheet_name)


构造分公司, 项目部, 项目三级组织机构

In [13]:
def industry_org(df_):
  industry_id = 1001
  df = df_.rename(columns={
    '机构id': 'id',
    '机构名称': 'org_name',
    '上级id': 'pid',
    '上级机构名称': 'p_org_name'
  })
  # branch
  df_branch = df.loc[df['pid'] == industry_id]
  # project department
  df_dept = pd.merge(
      df, df_branch[['id']], left_on='pid', right_on='id', suffixes=('', '_y'))
  df_industry = pd.merge(df_branch, df_dept,
                        left_on='id', right_on='pid', suffixes=('_branch', '_dept'))
  df_industry = df_industry[['id_branch',
                            'org_name_branch',
                            'id_dept',
                            'org_name_dept']]
  
  return df_industry


In [15]:
industry_org(tp_dfs['org'])

,id_branch,org_name_branch,id_dept,org_name_dept
0,1005,园区运营中心,1436204,北京产业创新中心
1,1005,园区运营中心,1436205,价值工厂
...,...,...,...,...
31,1435224,产园-武汉公司,1434223,高新网谷
32,1435225,产园-青岛公司,1435226,蓝湾网谷


In [21]:
def project_org(df_left, df_right):
    df_projects = df_right.rename(columns={
        'ORGAN_ID': 'org_id',
        '项目名称': 'project_name',
        '上级机构id': 'pid',
        '上级机构名称': 'p_org_name'
    })
    df_all = pd.merge(df_left, df_projects, left_on='id_dept', right_on='pid')
    df_all = df_all[['id_branch', 'org_name_branch',
                    'id_dept', 'org_name_dept', 'org_id']]
    df_all = df_all.rename(columns={'id_branch': 'branch_id',
                                    'org_name_branch': 'branch_name',
                                    'id_dept': 'dept_id',
                                    'org_name_dept': 'dept_name',
                                    'org_id': 'project_id'
                                    })
    return df_all


In [23]:
project_org(industry_org(tp_dfs['org']), tp_dfs['projects'])

,branch_id,branch_name,dept_id,dept_name,project_id
0,1005,园区运营中心,1436204,北京产业创新中心,1435203
1,1005,园区运营中心,1436205,价值工厂,1413262
...,...,...,...,...,...
93,1435225,产园-青岛公司,1435226,蓝湾网谷,1421248
94,1435225,产园-青岛公司,1435226,蓝湾网谷,1421249


In [4]:
lp = '2022-03-11'
tp = '2022-03-18'
# lp: last period
lp_filename = f'non-compliant_contracts-{lp}.xlsx'
lp_sheet_name = 'Sheet1'

# tp: this period
tp_filename = f'non-compliant_contracts-{tp}.xlsx'
tp_sheet_name = 'Sheet1'

# common
key_fields = ['合同编号']


## 读取上期数据


In [5]:
lp_path = data_dir / lp_filename
lp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-03-11.xlsx')

In [6]:
lp_db = pd.read_excel(lp_path, lp_sheet_name)
# deduplication based on 'contract number' field
lp_db = lp_db.drop_duplicates(subset=key_fields)
lp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-重庆公司,金山意库,1412212,金山意库,9号楼-场地租赁,jsyk-2022-02-0660,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
368,369,产园-深圳公司,蛇口网谷,1431202,万融大厦,万融大厦C座7层-702,wrds-2021-03-1031,深圳市万融大厦管理有限公司,深圳市敢为软件技术有限公司,应结未结,已终止未结算
372,373,园区运营中心,南海意库-商业,1433221,南海意库-商业,6栋1层-110,nhyk-sy-2021-12-1131,招商局蛇口工业区控股股份有限公司,深圳剪刀侠美发管理有限公司,应结未结,已终止未结算


## 读取本期数据


In [7]:
tp_path = data_dir / tp_filename
tp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-03-18.xlsx')

In [12]:
tp_db = pd.read_excel(tp_path, tp_sheet_name)
tp_db = tp_db.drop_duplicates(subset=key_fields)
tp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-重庆公司,金山意库,1412212,金山意库,9号楼-场地租赁,jsyk-2022-02-0660,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
466,467,产园-深圳公司,蛇口网谷,1431202,万融大厦,万融大厦C座7层-702,wrds-2021-03-1031,深圳市万融大厦管理有限公司,深圳市敢为软件技术有限公司,应结未结,已终止未结算
471,472,园区运营中心,南海意库-商业,1433221,南海意库-商业,6栋1层-110,nhyk-sy-2021-12-1131,招商局蛇口工业区控股股份有限公司,深圳剪刀侠美发管理有限公司,应结未结,已终止未结算


### 过滤增量数据中的公寓数据(临时)


In [13]:
tp_db = tp_db.loc[tp_db['项目名称'] != '东湖网谷公寓']
tp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-重庆公司,金山意库,1412212,金山意库,9号楼-场地租赁,jsyk-2022-02-0660,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
466,467,产园-深圳公司,蛇口网谷,1431202,万融大厦,万融大厦C座7层-702,wrds-2021-03-1031,深圳市万融大厦管理有限公司,深圳市敢为软件技术有限公司,应结未结,已终止未结算
471,472,园区运营中心,南海意库-商业,1433221,南海意库-商业,6栋1层-110,nhyk-sy-2021-12-1131,招商局蛇口工业区控股股份有限公司,深圳剪刀侠美发管理有限公司,应结未结,已终止未结算


## 提取增量数据

从本期数据中提取本期增量数据, 为了分析本期增量情况.

使用 `pandas` 做这种操作稍微有些复杂, 基本思路如下:

1. 使用 `left join` 模式将本期数据与上期数据进行 `merge`, 本期数据作为 `left`
2. 在使用 `merge` 函数时, 需要设置 `indicator` 参数为 `True`. 这样就可以在 `merge` 的结果表中增加一列用于表明 `merge` 的方式
3. 利用 `indicator` 来创建一个 Boolean indexing, 从而可以提取出增量数据


In [14]:
db_all = tp_db.merge(
    lp_db,
    on=key_fields,
    how='left',
    indicator=True,
    suffixes=(None, '_y')
)
incremental_flag = db_all['_merge'] == 'left_only'
incremental_db = db_all[tp_db.columns][incremental_flag]
incremental_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
2,3,产园-重庆公司,金山意库,1412212,金山意库,3号楼-3栋室外部分,jsyk-2022-03-0665,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
21,22,产园-深圳公司,光明科技园,1413273,招商局光明科技园,二期研发楼A1栋11层-A1-1101-2,招光加22A005,招商局光明科技园有限公司,深圳上泰生物工程有限公司,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
309,422,产园-杭州公司,信雅达创库,1421228,A1招商创库,A2信雅达国际1幢43层-A10房间号,A1zsck-2021-09-1531,杭州信雅达置业有限公司,杭州小力网络科技有限公司,应结未结,已终止未结算
316,432,产园-杭州公司,信雅达创库,1421228,A1招商创库,A2信雅达国际1幢43层-A209,A1zsck-2021-12-1609,杭州信雅达置业有限公司,林鹏,应结未结,已终止未结算


## 分析本期数据

分析思路:

1. 根据 `['分公司', '项目部', '情况']` 分组, 计算数量
2. 分组小计
3. 按分组小计倒排序


In [15]:
def analyze(_df):
    tp_grp = _df.groupby(['分公司', '项目部', '情况'])
    tp_grp_unstacked = tp_grp['organ_id'].count().unstack([-1, -2])
    subtotal = '--小计--'
    tp_grp_unstacked[('', subtotal)] = \
        tp_grp_unstacked.apply(lambda s: s.sum(), axis=1)

    tp_grp_stacked = tp_grp_unstacked.sort_values(
        ('', subtotal), ascending=False).stack([-1, -2]).to_frame('数量')
    tp_grp_stacked.loc[('总计', '', '')] = tp_grp['organ_id'].count().sum()

    return tp_grp_stacked.astype('int32')


In [16]:
analyze(tp_db)


数量
分公司     项目部    情况       
产园-深圳公司 --小计--       115
        光明科技园  倒签     26
...                  ...
产园-青岛公司 蓝湾网谷   应结未结    2
总计                   327

[47 rows x 1 columns]

## 分析增量数据


In [17]:
analyze(incremental_db)


数量
分公司     项目部    情况      
产园-深圳公司 --小计--       25
        光明科技园  倒签    12
...                  ..
产园-南京公司 高铁网谷   应结未结   1
总计                   42

[18 rows x 1 columns]

## 导出下发数据


In [18]:
output_dir_name = 'output'
out_dir = Path.cwd() / output_dir_name
if not out_dir.exists():
    out_dir.mkdir()

out_filename = f'{tp}-租赁平台-合同规范性检查（下发）.xlsx'

out_path = out_dir / out_filename

with pd.ExcelWriter(out_path) as writer:
    analyze(tp_db).reset_index().to_excel(writer, sheet_name='全量统计')
    analyze(incremental_db).reset_index().to_excel(writer, sheet_name='增量统计')
    tp_db.reset_index().to_excel(writer, sheet_name='全量数据')
    incremental_db.reset_index().to_excel(writer, sheet_name='增量数据')
